In [1]:
# Import the required libraries and dependencies
import requests
import pandas as pd
import talib
import requests
import numpy as np

from pathlib import Path
from itertools import compress
from matplotlib import pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc

#### Download BTC data and create BTC dataframe.

In [2]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    Path('../Resources/BTCUSD_1h.csv'),
    index_col="Date")
df.tail()

,Open,Close,High,Low,Volume
Date,,,,,
2021-11-17 13:00:00,60514.231682,60207.000000,60639.0,60207.0,476.771475
2021-11-17 14:00:00,60207.000000,59724.666301,60388.0,59724.0,824.163709
2021-11-17 15:00:00,59724.000000,59708.630836,59859.0,59483.0,500.389504
2021-11-17 16:00:00,59728.000000,60418.000000,60480.0,59491.0,398.163219
2021-11-17 17:00:00,60418.000000,60069.000000,60439.0,60031.0,109.235020


# Identify Candlesticks Pattern
* Based on the TA-Lib module created by Thomas Bulkowsi’s
* Source Code: https://github.com/CanerIrfanoglu/medium/blob/master/candle_stick_recognition/identify_candlestick.py

In [3]:
# extract OHLC 
op = df['Open'].astype(float)
hi = df['High'].astype(float)
lo = df['Low'].astype(float)
cl = df['Close'].astype(float) 


candle_names = talib.get_function_groups()['Pattern Recognition']
engulfing_candle = talib.CDLENGULFING(op, hi, lo, cl)
# create columns for each candle
for candle in engulfing_candle:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = engulfing_candle #getattr(talib, candle)(op, hi, lo, cl)
    
df.head()

,Open,Close,High,Low,Volume,0,100,-100
Date,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0


In [4]:
df['entry'] = np.where(df[0]>0,1,0)
df.head()

,Open,Close,High,Low,Volume,0,100,-100,entry
Date,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0,0


In [7]:
df.head()

,Open,Close,High,Low,Volume,0,100,-100,entry
Date,,,,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0,0,0,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,0,0,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,100,100,100,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0,0,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0,0,0


In [11]:
df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'entry']]
df.head()

,Open,Close,High,Low,Volume,entry
Date,,,,,,
2017-01-01 07:00:00,966.48,965.81,967.41,965.80,27.489011,0
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0


In [13]:
df['1h%'] = df['Close'].pct_change()
df['vol_pct_chg'] = df['Volume'].pct_change()
df = df.dropna()
df

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg
Date,,,,,,,,
2017-01-01 08:00:00,965.810000,965.460000,965.86,962.60,149.617696,0,-0.000362,4.442819
2017-01-01 09:00:00,965.410000,965.980000,966.57,965.37,32.288503,1,0.000539,-0.784193
2017-01-01 10:00:00,965.990000,977.010000,978.02,965.99,1061.834778,0,0.011418,31.885848
2017-01-01 11:00:00,977.710000,978.160000,978.89,975.05,712.321636,0,0.001177,-0.329160
2017-01-01 12:00:00,977.110000,973.320000,978.59,970.85,292.530740,0,-0.004948,-0.589328
...,...,...,...,...,...,...,...,...
2021-11-17 13:00:00,60514.231682,60207.000000,60639.00,60207.00,476.771475,0,-0.005077,0.422752
2021-11-17 14:00:00,60207.000000,59724.666301,60388.00,59724.00,824.163709,0,-0.008011,0.728635
2021-11-17 15:00:00,59724.000000,59708.630836,59859.00,59483.00,500.389504,0,-0.000268,-0.392852


In [ ]:
# # Assign value 0,1,-1 for No Pattern, Bull Pattern, and Bear Pattern
# df['candlestick_pattern'] = np.nan
# df['candlestick_index'] = np.nan
# for index, row in df.iterrows():

#     # bull pattern 100 or 200
#     if any(row[candle_names].values > 0):
#         pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bull'
#         df.loc[index, 'candlestick_pattern'] = pattern
#         df.loc[index, 'candlestick_index'] = 1
#     # bear pattern -100 or -200
#     elif any(row[candle_names].values < 0):
#         pattern = list(compress(row[candle_names].keys(), row[candle_names].values != 0))[0] + '_Bear'
#         df.loc[index, 'candlestick_pattern'] = pattern
#         df.loc[index, 'candlestick_index'] = -1
#     # no pattern
#     else:
#         df.loc[index, 'candlestick_pattern'] = "NO PATTERN"
#         df.loc[index, 'candlestick_index'] = 0

# df = df[['Open', 'Close', 'High', 'Low', 'Volume', 'candlestick_pattern', 'candlestick_index']]
# df['1h%'] = df['Close'].pct_change().dropna()
# df

### Integrating Technical Indicators
#### Simple Moving Average (SMA)

In [14]:
from ta.trend import SMAIndicator

# Add Simple Moving Average (SMA) indicators
df["sma7h"] = SMAIndicator(close=df["Close"], window=7, fillna=True).sma_indicator()
df["sma9h"] = SMAIndicator(close=df["Close"], window=9, fillna=True).sma_indicator()
df["sma20h"] = SMAIndicator(close=df["Close"], window=20, fillna=True).sma_indicator()
df["sma200d"] = SMAIndicator(close=df["Close"], window=4800, fillna=True).sma_indicator()

# Review the DataFrame
df

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,sma7h,sma9h,sma20h,sma200d
Date,,,,,,,,,,,,
2017-01-01 08:00:00,965.810000,965.460000,965.86,962.60,149.617696,0,-0.000362,4.442819,965.460000,965.460000,965.460000,965.460000
2017-01-01 09:00:00,965.410000,965.980000,966.57,965.37,32.288503,1,0.000539,-0.784193,965.720000,965.720000,965.720000,965.720000
2017-01-01 10:00:00,965.990000,977.010000,978.02,965.99,1061.834778,0,0.011418,31.885848,969.483333,969.483333,969.483333,969.483333
2017-01-01 11:00:00,977.710000,978.160000,978.89,975.05,712.321636,0,0.001177,-0.329160,971.652500,971.652500,971.652500,971.652500
2017-01-01 12:00:00,977.110000,973.320000,978.59,970.85,292.530740,0,-0.004948,-0.589328,971.986000,971.986000,971.986000,971.986000
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-17 13:00:00,60514.231682,60207.000000,60639.00,60207.00,476.771475,0,-0.005077,0.422752,60046.752999,59953.696777,59872.711649,45962.100038
2021-11-17 14:00:00,60207.000000,59724.666301,60388.00,59724.00,824.163709,0,-0.008011,0.728635,60047.562470,59965.437477,59863.354399,45962.582677
2021-11-17 15:00:00,59724.000000,59708.630836,59859.00,59483.00,500.389504,0,-0.000268,-0.392852,60060.509733,59973.396459,59837.312487,45963.079267


#### Bollinger Bands

In [15]:
from ta.volatility import BollingerBands

# Add Bollinger Bands indicator
indicator_bb = BollingerBands(close=df["Close"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

# Review the DataFrame
df.head(20)

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,sma7h,sma9h,sma20h,sma200d,bb_bbm,bb_bbh,bb_bbl
Date,,,,,,,,,,,,,,,
2017-01-01 08:00:00,965.81,965.46,965.86,962.60,149.617696,0,-0.000362,4.442819,965.460000,965.460000,965.460000,965.460000,NaN,NaN,NaN
2017-01-01 09:00:00,965.41,965.98,966.57,965.37,32.288503,1,0.000539,-0.784193,965.720000,965.720000,965.720000,965.720000,NaN,NaN,NaN
2017-01-01 10:00:00,965.99,977.01,978.02,965.99,1061.834778,0,0.011418,31.885848,969.483333,969.483333,969.483333,969.483333,NaN,NaN,NaN
2017-01-01 11:00:00,977.71,978.16,978.89,975.05,712.321636,0,0.001177,-0.329160,971.652500,971.652500,971.652500,971.652500,NaN,NaN,NaN
2017-01-01 12:00:00,977.11,973.32,978.59,970.85,292.530740,0,-0.004948,-0.589328,971.986000,971.986000,971.986000,971.986000,NaN,NaN,NaN
2017-01-01 13:00:00,973.98,976.89,978.51,973.75,82.366898,0,0.003668,-0.718433,972.803333,972.803333,972.803333,972.803333,NaN,NaN,NaN
2017-01-01 14:00:00,976.58,978.45,978.49,976.00,115.830580,0,0.001597,0.406276,973.610000,973.610000,973.610000,973.610000,NaN,NaN,NaN
2017-01-01 15:00:00,978.50,981.96,982.00,976.46,459.671480,0,0.003587,2.968481,975.967143,974.653750,974.653750,974.653750,NaN,NaN,NaN
2017-01-01 16:00:00,981.97,996.29,999.00,976.24,4757.045348,0,0.014593,9.348794,980.297143,977.057778,977.057778,977.057778,NaN,NaN,NaN


In [16]:
df.dropna(inplace=True)
df.head()

C:\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Open,Close,High,Low,Volume,entry,1h%,vol_pct_chg,sma7h,sma9h,sma20h,sma200d,bb_bbm,bb_bbh,bb_bbl
Date,,,,,,,,,,,,,,,
2017-01-02 03:00:00,1011.5,1013.0,1015.0,1009.0,737.045368,0,0.001483,0.707852,1005.025714,1004.300000,991.2765,991.276500,991.2765,1020.548664,962.004336
2017-01-02 04:00:00,1014.0,1015.0,1018.0,1010.5,423.309560,0,0.001974,-0.425667,1006.511429,1006.020000,993.7535,992.406190,993.7535,1022.241744,965.265256
2017-01-02 05:00:00,1014.1,1011.4,1016.8,1008.9,235.217866,0,-0.003547,-0.444336,1007.468571,1006.842222,996.0245,993.269545,996.0245,1022.462288,969.586712
2017-01-02 06:00:00,1012.0,1016.7,1016.9,1010.4,238.331585,0,0.005240,0.013238,1009.998571,1008.186667,998.0090,994.288261,998.0090,1024.398177,971.619823
2017-01-02 07:00:00,1016.6,1018.2,1018.9,1014.1,401.187665,0,0.001475,0.683317,1012.600000,1009.686667,1000.0110,995.284583,1000.0110,1026.147094,973.874906


In [20]:
df.to_csv(Path('../../../LSTM testing/test.csv'))